## Título: Changer Sentinel-2 metadata for make it readable for snappy 9
## Fecha: 06/04/2025
## Autor: Gustavo V. Diaz
[Ref link unsolved](https://forum.step.esa.int/t/sen2cor-supporting-processing-baseline-05-11/42828)

In [1]:
import os
import zipfile

# Módulo para modificar xml para hacerlo compatible con snappy 9
import xml.etree.ElementTree as ET

In [2]:
# path2prods = r'/src/Output/SMM_vacas_2025_test_3_20250405T183331/tmp/'
# display(os.listdir(path2prods))
path2prods = r'/src/Output/SMM_vacas_2025_test_3_20250405T183331/tmp/'
# Get all files in the directory with .zip extension
zip_files = [file for file in os.listdir(path2prods) 
             if file.endswith('.zip') and os.path.isfile(os.path.join(path2prods, file))]
display(zip_files)

['S2A_MSIL2A_20250112T134701_N0511_R024_T21HUA_20250112T183356.SAFE.zip',
 'S2B_MSIL2A_20241228T134709_N0511_R024_T21HUA_20241228T172745.SAFE.zip',
 'S2B_MSIL2A_20241208T134709_N0511_R024_T21HUA_20241208T190915.SAFE.zip',
 'S2A_MSIL2A_20241203T134701_N0511_R024_T21HUA_20241203T183351.SAFE.zip',
 'S2A_MSIL2A_20241113T134701_N0511_R024_T21HUA_20241113T183442.SAFE.zip',
 'S2A_MSIL2A_20240924T134711_N0511_R024_T21HUA_20240924T190249.SAFE.zip',
 'S2B_MSIL2A_20241029T134709_N0511_R024_T21HUA_20241029T173341.SAFE.zip',
 'S2A_MSIL2A_20240726T134711_N0511_R024_T21HUA_20240727T091206.SAFE.zip',
 'S2A_MSIL2A_20240914T134701_N0511_R024_T21HUA_20240914T202753.SAFE.zip']

In [3]:
def list_xml_files(zip_path):
    """List all .xml files in a ZIP archive"""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        xml_files = [f for f in zip_ref.namelist() if f.lower().endswith('.xml')]
        return xml_files

In [3]:
# choose one file and get it content that has the '*.xml' extension
file4dev = zip_files[0]
print(file4dev)
# I try to extract the file MTD_MSIL2A.xml that exist in every Sentinel-2 product
# Search every xml
from esa_snappy import ProductIO
path4dev = os.path.join(path2prods, file4dev)
# xml_list = list_xml_files(path4dev)
# display(xml_list)
# file2extract = xml_list[-1]
product = ProductIO.readProduct(path4dev)

S2A_MSIL2A_20250112T134701_N0511_R024_T21HUA_20250112T183356.SAFE.zip


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/esa-snap/snap/modules/org.esa.snap.snap-netcdf/org-slf4j/slf4j-simple.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/esa-snap/snap/modules/ext/org.esa.snap.snap-netcdf/org-slf4j/slf4j-simple.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.SimpleLoggerFactory]
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: eu.esa.opt.dataio.s2.ortho.S2OrthoProductReaderPlugIn: Building product reader - EPSG:32721




In [4]:
# Procesamiento de datos
width = product.getSceneRasterWidth()
print(f"Ancho: {width} px\n")
height = product.getSceneRasterHeight()
print(f"Alto: {height} px\n")
prod_name = product.getName()
print(f"Nombre: {prod_name}\n")
band_names = product.getBandNames()
# print(f"Nombre de bandas: {band_names}")
# print("Band names: {}".format(", ".join(band_names)))
str_band_n = ", ".join(band_names).split(", ")
print(f"Cantidad de bandas: {len(str_band_n)}\n")
# display(len(str_band_n), str_band_n)
crs_raster = product.getSceneCRS()
geocod_raster = product.getSceneGeoCoding()

Ancho: 10980 px

Alto: 10980 px

Nombre: S2A_MSIL2A_20250112T134701_N0511_R024_T21HUA_20250112T183356

Cantidad de bandas: 158



In [5]:
def find_xml_files(start_dir):
    """
    Recursively finds all .xml files in a directory and its subdirectories
    Returns list of absolute file paths
    """
    xml_files = []
    for root, dirs, files in os.walk(start_dir):
        for file in files:
            if file.lower().endswith('.xml'):
                full_path = os.path.join(root, file)
                xml_files.append(full_path)
    return xml_files

In [6]:
# Extract just MTD_MSIL2A.xml and rewrite it as Deepseek told me.
# Extract a specific file
with zipfile.ZipFile(path4dev, 'r') as zip_ref:
    zip_ref.extract(file2extract, '.')

# listo archivo xml en directorio base
xml_list_extracted = find_xml_files('.')

display(xml_list_extracted)

['./S2A_MSIL2A_20250112T134701_N0511_R024_T21HUA_20250112T183356.SAFE/MTD_MSIL2A.xml']

In [7]:
# Copy original file to save the original info
def copy_file_with_new_name(src_file, extension):
    """
    Copies a file to a new location with a different name
    
    Args:
        src_file (str): Path to source file
        dest_dir (str): Destination directory
        new_name (str): New filename (without path)
    """
    # Create destination directory if it doesn't exist
    # os.makedirs(dest_dir, exist_ok=True)
    folder2copy = os.path.dirname(src_file)
    orig_file_name = os.path.basename(src_file).split('.')
    new_name = orig_file_name[0] + extension + orig_file_name[1]
    # new_path = os.path.join(folder2copy, new_name)
    
    # Build full destination path
    dest_file = os.path.join(folder2copy, new_name)
    
    # Copy the file
    with open(src_file, 'rb') as src, open(dest_file, 'wb') as dst:
        dst.write(src.read())
    
    return dest_file

def create_path(src_file, extension):
    # Create destination directory if it doesn't exist
    # os.makedirs(dest_dir, exist_ok=True)
    folder2copy = os.path.dirname(src_file)
    orig_file_name = os.path.basename(src_file).split('.')
    new_name = orig_file_name[0] + extension + orig_file_name[1]
    # new_path = os.path.join(folder2copy, new_name)
    
    # Build full destination path
    dest_file = os.path.join(folder2copy, new_name)
    
    return dest_file



In [8]:
pathched_path = create_path(xml_list_extracted[0], '_patched')

In [9]:
# Load the XML file
tree = ET.parse(xml_list_extracted[0])
root = tree.getroot()

# Add legacy <General_Info> section (required by SNAP 9)
if not root.find('General_Info'):
    general_info = ET.SubElement(root, 'General_Info')
    product_info = ET.SubElement(general_info, 'Product_Info')
    
    # Copy critical metadata from new format
    sensing_time = root.find('.//SENSING_TIME').text
    product_uri = root.find('.//PRODUCT_URI').text
    
    ET.SubElement(product_info, 'PRODUCT_START_TIME').text = sensing_time
    ET.SubElement(product_info, 'PRODUCT_URI').text = product_uri

# Save the modified file
tree.write(pathched_path, encoding='UTF-8', xml_declaration=True)

AttributeError: 'NoneType' object has no attribute 'text'